In [1]:
import boto3
from pprint import pprint 

import boto3_helper

import os



In [2]:
# $Env:AWS_PROFILE="lcl14"
# $Env:AWS_SECRET_NAME="secret_manager_aws_lcl14"
# $Env:DD_SECRET_NAME="secret_manager_datadog_lcl14"
# $Env:REGION="ap-northeast-2"  

os.environ['AWS_PROFILE']     = 'lcl14'
os.environ['DD_SECRET_NAME']  = 'secret_manager_datadog_lcl14'
os.environ['AWS_SECRET_NAME'] = 'secret_manager_aws_lcl14'  # 시크릿의 이름을 지정하세요.
os.environ['REGION']          = 'ap-northeast-2'  # 시크릿 매니저가 있는 지역을 지정하세요.

In [3]:
print(os.getenv('REGION'))
print(os.getenv('AWS_SECRET_NAME'))

ap-northeast-2
secret_manager_aws_lcl14


In [4]:

boto3_helper.init_aws_session( )

Session(region_name='ap-northeast-2')

In [5]:
# DynamoDB 클라이언트 생성
dynamodb = boto3.client('dynamodb')

# 테이블 이름
table_name = 'dynamodb_system_check_lcl14'



In [11]:
# 인덱스 이름
index_name = 'check_dtm-index'

# 인덱스 키 스키마
key_schema = [
    {'AttributeName': 'resource_id', 'KeyType': 'HASH'},  # HASH 키
    {'AttributeName': 'check_dtm', 'KeyType': 'RANGE'}  # RANGE 키
]

# 인덱스 프로젝션
projection = {'ProjectionType': 'ALL'}

# 인덱스 생성 요청
dynamodb.update_table(
    TableName=table_name,
    AttributeDefinitions=[
        {'AttributeName': 'resource_id', 'AttributeType': 'S'},
        {'AttributeName': 'check_dtm', 'AttributeType': 'S'}
    ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': index_name,
                'KeySchema': key_schema,
                'Projection': projection
            }
        }
    ]
)

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'check_dtm',
    'AttributeType': 'S'},
   {'AttributeName': 'resource_id', 'AttributeType': 'S'}],
  'TableName': 'dynamodb_system_check_lcl14',
  'KeySchema': [{'AttributeName': 'resource_id', 'KeyType': 'HASH'},
   {'AttributeName': 'check_dtm', 'KeyType': 'RANGE'}],
  'TableStatus': 'UPDATING',
  'CreationDateTime': datetime.datetime(2023, 6, 27, 18, 46, 51, 613000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 3892382,
  'ItemCount': 2818,
  'TableArn': 'arn:aws:dynamodb:ap-northeast-2:123456789012:table/dynamodb_system_check_lcl14',
  'TableId': '4c760477-275b-4e55-ae43-aa83b790bfae',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(2023, 6, 27, 18, 46, 51, 613000, tzinfo=tzlocal())},
  'GlobalSecondaryIndexes': [{'IndexName': 'resource_id-check

In [18]:
# 금일 날짜 ('2023/07/04')
date        = '2023/07/04'
resource_id = '*'


# Query 파라미터 설정
query_params = {
    'TableName': table_name,
    # 'IndexName': 'resource_id-check_dtm-index',  # 인덱스 이름
    # 'IndexName': 'resource_id-check_dtm-index',  # 인덱스 이름
    'IndexName': index_name,  # 인덱스 이름 
    'KeyConditionExpression': 'resource_id = :resource_id and begins_with(check_dtm, :date)',
    # 'KeyConditionExpression': 'begins_with(check_dtm, :date)',
    'ExpressionAttributeValues': {
        ':resource_id': {'S': resource_id},  # resource_id 값
        ':date': {'S': date}  # 날짜 값
    },
    'ScanIndexForward': False,  # 내림차순 정렬 (가장 최근 값부터 가져옴)
    'Limit': 1  # 가져올 아이템 개수
}

pprint(query_params)

# pprint(**query_params)

{'ExpressionAttributeValues': {':date': {'S': '2023/07/04'},
                               ':resource_id': {'S': '*'}},
 'IndexName': 'check_dtm-index',
 'KeyConditionExpression': 'resource_id = :resource_id and '
                           'begins_with(check_dtm, :date)',
 'Limit': 1,
 'ScanIndexForward': False,
 'TableName': 'dynamodb_system_check_lcl14'}


In [19]:
# 쿼리 실행
response = dynamodb.query(**query_params)

pprint(response)



{'Count': 0,
 'Items': [],
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '39',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Tue, 04 Jul 2023 12:38:00 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3413411624',
                                      'x-amzn-requestid': '1RPO7IOQOMB052M4M2G17F024NVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '1RPO7IOQOMB052M4M2G17F024NVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
 'ScannedCount': 0}


In [20]:
# 결과 처리
for item in response['Items']:
    resource_id = item['resource_id']['S']
    check_dtm = item['check_dtm']['S']
    print(f"resource_id: {resource_id}, check_dtm: {check_dtm}")